<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

## User-local Environment Creation
**This notebook must be run in the base kernel.**

This notebook provides the means by which additional conda environments that support specific science data packages are created local to the user (i.e.'user-local'). The implication of user-local installations is that they are retained across logins and server restarts (since the ~jovyan directory is retained and remounted each time). It also permits the user to customize the environment, again with those changes persisted across sessions.

This notebook provides the steps to create the following user-local environments:
- ISCE2/ISCE3
- MintPy/ARIATools
- PLAnT

Each environment also contains the PCM APIs (pele-client, otello, osaka) and tools (notebook-pge-wrapper).

#### Install kernda
Kernda will be used to make ipython aware of the environments we create.

In [ ]:
pip install kernda

### What environments are currently available?
Come back and re-execute this cell any time to see what has and hasn't been created. Those found under /home/jovyan are user-local while others are global.

In [ ]:
%conda env list

### Configure conda
This will make conda recognize directories under ~/.local/envs as their environment name. This only needs to be run once, ever.

In [ ]:
%%bash
mkdir -p ~/.local/envs
conda config --append envs_dirs ~/.local/envs

# Solid Earth

### ISCE2 + MintPy + ARIA Tools + Solid Earth ATBDs


In [ ]:
%%bash

env_base="/home/jovyan/.local/envs/solid_earth_atbd"
local_base="/home/jovyan/.local"

# (re)create the ISCE2 environment that includes the packages required for Solid Earth ATBDs
mamba env create -f env_files/isce_conda_env.yml -p ${env_base} 
# for now, have to use a slightly customized ATBD dependency yml file which specifies the correct version of dem_stitcher
mamba env update -f env_files/solid_earth_atbd.yml -p ${env_base}


# clone the Solid Earth ATBD repo into home directory FIX THIS
atbd_path=~/ATBD
conda env config vars set -n solid_earth_atbd PYTHONPATH=${atbd_path}:${PYTHONPATH}

# ISCE set-up
isce_package=${env_base}/lib/python3.9/site-packages/isce

# install the license
popd
tar zxf support_files/stanford_components.tgz -C ${isce_package}

# Variables for ISCE set-up
source /opt/conda/bin/activate ${env_base}
conda env config vars set -n solid_earth_atbd PATH=${env_base}/bin:${isce_package}/applications:${PATH}
conda env config vars set -n solid_earth_atbd PYTHONPATH=${isce_package}:${PYTHONPATH}

# MintPy set-up

# (re)create the MintPy/ARIATools environment
mamba env update -f env_files/mintpy_conda_env.yml -p ${env_base} 

# make the base mintpy files executable
chmod 755 ${env_base}/lib/python*/site-packages/mintpy/*.py

# install/build ARIA-tools
aria_install_dir=${local_base}/tools/ARIA-tools
if [ ! -d ${aria_install_dir} ] && echo "Checking out ARIA-tools"
then
    # add -b <branch/tag> to the following command to install one other than dev
    git clone https://github.com/aria-tools/ARIA-tools ${aria_install_dir}
fi

# even if the directory previously exists, this has to be done since the environment is being recreated
pushd ${aria_install_dir}
git pull
mamba run -n solid_earth_atbd python setup.py build
mamba run -n solid_earth_atbd python setup.py install
popd
# update the ipykernel to recognize the new kernel
conda run -n solid_earth_atbd python -m ipykernel install --user --name solid_earth_atbd --display-name "solid_earth_atbd"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
conda run -n solid_earth_atbd kernda --env-dir ${env_base} --display-name solid_earth_atbd -o ~/.local/share/jupyter/kernels/solid_earth_atbd/kernel.json

# Additional ARIA-tools set-up
source /opt/conda/bin/activate ${env_base}
conda env config vars set -n solid_earth_atbd PATH=${aria_install_dir}:${PATH}
conda env config vars set -n solid_earth_atbd PYTHONPATH=${aria_install_dir}:${PYTHONPATH}

### MintPy + Solid Earth ATBDs

In [ ]:
%%bash

env_base="/home/jovyan/.local/envs/mintpy_dev"
local_base="/home/jovyan/"

# Clone MintPy and install dependencies
pushd ${local_base}

# OPTIONAL: Clone the mintpy repo
git clone https://github.com/insarlab/MintPy.git # Comment this out if you have already done this once

#Pull the latest updates
git pull # Or pull the newest version if you have already installed 
mamba create --name mintpy_dev --file ${local_base}/MintPy/requirements.txt -c conda-forge
conda activate mintpy_dev
python -m pip install -e ${local_base}/MintPy
conda install -c conda-forge sardem
popd

# Clone the Solid Earth ATBD repo and install dependencies
pushd ${local_base}

# OPTIONAL: Clone the Solid Earth ATBD repo into home directory, or update it
git clone https://gitlab.com/nisar-science-algorithms/solid-earth/atbd-se-development.git
popd

# Or just pull the updates
atbd_path=${local_base}/atbd-se-development
pushd ${atbd_path}
git pull

mamba env update -n mintpy_dev -f env_files/solid_earth_atbd.yml -p ${env_base}
conda env config vars set -n mintpy_dev PYTHONPATH=${atbd_path}:${PYTHONPATH}
popd

# update the ipykernel to recognize the new kernel
conda run -n mintpy_dev python -m ipykernel install --user --name mintpy_dev --display-name "mintpy_dev"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
conda run -n mintpy_dev kernda --env-dir ${env_base} --display-name "mintpy_dev" -o ~/.local/share/jupyter/kernels/mintpy_dev/kernel.json

### Ecosystems ATBDs


In [ ]:
%%bash

env_base="/home/jovyan/.local/envs/ecosystems_atbd"

# Create the ecosystems_atbd environment that includes the packages required for Ecosystems ATBDs
mamba env create -f env_files/ecosystems_atbd.yml -p ${env_base} --force

# update the ipykernel to recognize the new kernel
conda run -n ecosystems_atbd python -m ipykernel install --user --name ecosystems_atbd --display-name "ecosystems_atbd"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
conda run -n ecosystems_atbd kernda --env-dir ${env_base} --display-name ecosystems_atbd -o ~/.local/share/jupyter/kernels/ecosystems_atbd/kernel.json


### Ecosystems ATBDs with SNAPPY and Python = 3.6

In [ ]:
%%bash

env_base="/home/jovyan/.local/envs/snappy_py36"

# Create the ecosystems_atbd environment that includes the packages required for Ecosystems ATBDs
mamba env create -f env_files/snappy_py36.yml -p ${env_base} --force

# update the ipykernel to recognize the new kernel
conda run -n snappy_py36 python -m ipykernel install --user --name snappy_py36 --display-name "snappy_py36"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
conda run -n snappy_py36 kernda --env-dir ${env_base} --display-name snappy_py36 -o ~/.local/share/jupyter/kernels/snappy_py36/kernel.json



In [ ]:
# download the snap linex installer
!pip install wget
import wget
url = 'https://download.esa.int/step/snap/9.0/installers/esa-snap_sentinel_unix_9_0_0.sh'
filename = wget.download(url, out='/home/jovyan/')

# install SNAP
!chmod +x esa-snap_sentinel_unix_9_0_0.sh

## see next steps to install SNAP through the terminal

To install SNAP and configure for Python, the following steps are required:

Within the Terminal, run the following:

- conda activate snappy_py35
- cd /home/jovyan/sds-ondemand/environments
- ./esa-snap_sentinel_unix_9_0_0.sh
- rm -r esa-snap_sentinel_unix_9_0_0.sh
    
**Follow the prompts, which are summarized below.**

This will install ESA SNAP on your computer.

**o**

Where should ESA SNAP be installed?

**[/home/jovyan/snap]**

Which components should be installed?

**[2,3,4,5]**

Create symlinks?

**n**

Configure SNAP for use with Python?

**y**

Python executable:

** /home/jovyan/.local/envs/snappy_py36/bin/python **

Run SNAP Desktop and configure Python?

**y**


### R environment

In [ ]:
%%bash

env_base="/home/jovyan/.local/envs/r_env"

# Create the ecosystems_atbd environment that includes the packages required for Ecosystems ATBDs
mamba env create -f env_files/r_env.yml -p ${env_base} --force

# update the ipykernel to recognize the new kernel
conda run -n r_env python -m ipykernel install --user --name r_env --display-name "r_env"
# now invoke kernda to modify the kernel file to ensure the kernel is invoked from within the conda environment
conda run -n r_env kernda --env-dir ${env_base} --display-name r_env -o ~/.local/share/jupyter/kernels/r_env/kernel.json


### Creating other user-local environments.

As shown above, the command used to create a conda environment is:

```conda env create <arguments>```

In the above cases, the ```-f``` switch is used to denote the input file which describes the environment's contents, including the source conda channels (e.g. default, conda-forge) and the environment prefix (in this case, subdirectories of /home/jovyan/.local/envs). The prefix is responsible for designating where the environment's contents will reside and if a subdirectory of the user (i.e. jovyan), the environment is user-local. Since the ~jovyan directory is persisted and remounted for each jupyter login, these environments also persist and do not have to be created each time you log into jupyter.

The --force flag in the command lines used above forces conda to recreate (rather than just update) the environment from scratch.

For clarity and organization, it's best to maintain the local environments in the same user subdirectory.

To create a new user-local environment from scratch, use the following command:

```conda env create -p /home/jovyan/.local/envs/<env-name>```


To start with a copy of another environment (such as pcm_tools), omit the ```env``` and use the ```--clone``` command line option:

```conda create -p /home/jovyan/.local/envs/<env-name> --clone pcm_tools```

Once created, switch to the new environment accordingly (this is only applicable in terminal windows):

```conda activate <env-name>```

Currently active environments and their prefix can be listed by:

```conda env list```

An environment can be removed by:

```conda env remove -n <env name>``` - to remove by name

or

```conda env remove -p <prefix>``` - to remove by prefix

Finally, to make ipython aware of the environment, invoke ipykernel:

```conda run -n <env name> python -m ipykernel install --user --name <env name> --display-name <env name>```

then run kernda:

```conda run -n <env name> kernda --env-dir ~/.local/share/<env name> --display-name <env name> \
         -o ~/.local/share/jupyter/kernels/<env name>/kernel.json```

Running kernda ensures that the kernel is launched from within the associated conda environment. This means that any shell commands invoked within the notebook (e.g. using the %%bash magic) execute within that environment.

You will need to reload the jupyter landing page in order to see the environment under the ```New``` pull down.

<font size="1">This notebook is compatible with NISAR Jupyter Server Image v1.7.3 and above</font>